In [5]:
import pandas as pd
import numpy as np
import joblib
import time
import random
import warnings
import os

warnings.filterwarnings("ignore")

# --- Configuration ---
DATASET_CSV = 'refrigerator_unified_dataset.csv'
SCALER_FILE = 'refrigerator_multi_task_scaler.joblib'
SCREENER_MODEL_FILE = 'refrigerator_screener_model.joblib'
WHY_MODEL_FILE = 'refrigerator_diagnostician_why.joblib'
WHEN_MODEL_FILE = 'refrigerator_diagnostician_when.joblib'

SEQUENCE_TIMESTEPS = 336 # 14 days for refrigerators (14 * 24 readings)
MAE_DAYS = 2.87 # The MAE from your evaluation report

def run_final_refrigerator_simulation():
    """
    Runs the final live simulation using the complete two-stage AI agent for refrigerators.
    """
    # --- 1. Load All Models and Data ---
    print("--- Loading all models and data for the final simulation ---")
    try:
        scaler = joblib.load(SCALER_FILE)
        screener_model = joblib.load(SCREENER_MODEL_FILE)
        why_model = joblib.load(WHY_MODEL_FILE)
        when_model = joblib.load(WHEN_MODEL_FILE)
        df = pd.read_csv(DATASET_CSV, parse_dates=['timestamp'])
        print("✅ All files loaded successfully.")
    except Exception as e:
        print(f"❌ Error loading files: {e}")
        return

    # --- 2. Isolate a Real Failure Case ---
    print("\n--- Randomly selecting a real failure case from the dataset ---")
    failing_device_ids = df[df['failure_type'] != 0]['device_id'].unique()
    device_to_test = random.choice(failing_device_ids)
    device_df = df[df['device_id'] == device_to_test].sort_values(by='timestamp').reset_index(drop=True)

    failure_point_idx = device_df[device_df['days_until_failure'] < 1].index.min()
    test_window_start_idx = max(0, failure_point_idx - (30 * 24)) # Start story 30 days before failure

    test_story_df = device_df.iloc[test_window_start_idx:failure_point_idx]

    failure_map = {0: "Healthy", 1: "Compressor Failure", 2: "Thermostat Failure", 3: "Seal Failure"}
    ground_truth_failure_type_code = int(test_story_df['failure_type'].max())
    ground_truth_failure_type = failure_map[ground_truth_failure_type_code]

    print(f"Test case selected: '{device_to_test}'. The ground truth is a '{ground_truth_failure_type}'.")

    # --- 3. Run the Day-by-Day Simulation ---
    print("\n--- Starting Final Live Simulation ---")
    print("-" * 120)

    initial_healthy_data = device_df.iloc[test_window_start_idx - SEQUENCE_TIMESTEPS : test_window_start_idx]
    live_buffer = list(initial_healthy_data.to_dict('records'))

    last_prediction = "None"

    print(f"{'Date':<12} | {'True Days to Fail':<20} | {'AI Agent Report'}")
    print("-" * 120)

    for i in range(0, len(test_story_df), 24): # Step by 1 day (24 readings)
        day_data = test_story_df.iloc[i:i+24]
        if day_data.empty: continue

        for _, row in day_data.iterrows():
            live_buffer.append(row.to_dict())
            if len(live_buffer) > SEQUENCE_TIMESTEPS: live_buffer.pop(0)

        if len(live_buffer) != SEQUENCE_TIMESTEPS: continue

        current_date = day_data['timestamp'].iloc[-1]
        true_days_until_failure = day_data['days_until_failure'].iloc[-1]

        sequence_df = pd.DataFrame(live_buffer)
        feature_columns = ['temperature_current_c', 'compressor_on', 'door_open', 'defrost_cycle_active', 'ambient_temp_c', 'filter_life_percent']
        scaled_features = scaler.transform(sequence_df[feature_columns])
        flattened_sequence = scaled_features.reshape(1, -1)

        status_report = "Prediction: Normal Operation"

        # Stage 1: The Screener (using default 0.5 threshold as precision was already high)
        risk_score = screener_model.predict_proba(flattened_sequence)[:, 1][0]

        if risk_score > 0.5:
            # Stage 2: The Diagnostician
            predicted_reason_code = why_model.predict(flattened_sequence)[0]
            predicted_days = when_model.predict(flattened_sequence)[0]

            lower_bound = max(0, predicted_days - MAE_DAYS)
            upper_bound = predicted_days + MAE_DAYS

            predicted_reason = failure_map.get(predicted_reason_code, "Unknown")
            status_report = f"Prediction: Failure likely in {lower_bound:.1f} to {upper_bound:.1f} days. Predicted Cause: '{predicted_reason}'"
            last_prediction = predicted_reason

        print(f"{current_date.strftime('%Y-%m-%d'):<12} | {true_days_until_failure:<20.1f} | {status_report}")
        time.sleep(0.1)

    # --- 4. Final Verdict ---
    print("-" * 120)
    print(f"\n💥 DEVICE FAILURE OCCURRED 💥")
    print("\n--- FINAL TEST VERDICT ---")
    print(f"Ground Truth Failure Cause: '{ground_truth_failure_type}'")
    print(f"Model's Final Prediction:   '{last_prediction}'")

    if last_prediction == ground_truth_failure_type:
        print(f"\n✅ SUCCESS: The complete AI agent correctly identified the failure type.")
    else:
        print(f"\n❌ FAILURE: The AI agent misdiagnosed the failure type.")

if __name__ == "__main__":
    run_final_refrigerator_simulation()

--- Loading all models and data for the final simulation ---
✅ All files loaded successfully.

--- Randomly selecting a real failure case from the dataset ---
Test case selected: 'fridge_018'. The ground truth is a 'Thermostat Failure'.

--- Starting Final Live Simulation ---
------------------------------------------------------------------------------------------------------------------------
Date         | True Days to Fail    | AI Agent Report
------------------------------------------------------------------------------------------------------------------------
2023-10-18   | 30.0                 | Prediction: Normal Operation
2023-10-19   | 29.0                 | Prediction: Normal Operation
2023-10-20   | 28.0                 | Prediction: Normal Operation
2023-10-21   | 27.0                 | Prediction: Normal Operation
2023-10-22   | 26.0                 | Prediction: Normal Operation
2023-10-23   | 25.0                 | Prediction: Normal Operation
2023-10-24   | 24.0      